# MANUAL SKETCHBOOK

## PREPARACIÓN 

### Python 3.X

https://www.python.org/downloads/

### Librerias necesarias

- selenium
- utm
- requests
- pandas
- numpy

### Chrome-driver

https://chromedriver.storage.googleapis.com/index.html?path=89.0.4389.23/

## ORGANIZACIÓN DE LA BASE DE DATOS 

La realización de esto se hará en base al siguiente documento https://docs.gbif-uat.org/georeferencing-quick-reference-guide/1.0/en/georeferencing-quick-reference-guide-community-review-draft.en.pdf (punto 1.5)

En resumen, tendremos dos posibles casos de georreferenciación.

El primero corresponde a la existencia de un punto con coordenadas geograficas definidas, por lo que es necesario rellenar los siguientes campos
- verbatimLatitude
- verbatimLongitude
- verbatimSRS


## PROCESAMIENTO DE LA BASE DE DATOS

## AUTOMATIZACIÓN DE "GEOREFERENCING CALCULATOR"

## POST PROCESAMIENTO DE LA BASE DE DATOS

## EXPORTAR BASE DE DATOS

In [1]:
from GeoParser import *
import pandas as pd
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
data=pd.read_csv("setCoordinatesPeces.csv")
#data["geodeticDatum"]="(WGS84) World Geodetic System 1984"
#data["verbatimSRS"]="(WGS84) World Geodetic System 1984"
data["verbatimCoordinateSystem"]="decimal degrees"
#data["decimalLongitude"]=pd.NA
data["dynamicProperties"]=str({"textFieldMeasurementError": 30,"ChoiceDistUnits": "m", "ChoiceCoordSource": "GPS", "TextFieldExtent": 0})
#data.verbatimLatitude=data.verbatimLatitude.replace('"S"','" S')
#data.verbatimLongitude=data.verbatimLongitude.replace('O"',' "O')
#data.verbatimLongitude=data.verbatimLongitude.replace('W"',' "W')
data.head(2)

,Unnamed: 0,decimalLongitude,decimalLatitude,verbatimSRS,geodeticDatum,continent,country,countryCode,stateProvince,county,municipality,verbatimCoordinates,verbatimCoordinateSystem,dynamicProperties
0,1,-37.985806,-73.270090,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,-73.27009 -37.985806,decimal degrees,"{'textFieldMeasurementError': 30, 'ChoiceDistU..."
1,2,-37.962703,-73.250557,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,-73.250557 -37.962703,decimal degrees,"{'textFieldMeasurementError': 30, 'ChoiceDistU..."


In [ ]:
%%time
#Testing...
DBPrep(data).verbatim_coordinates_autocomplete()
DBPrep(data).verbatim_coordinate_system_autocomplete()
DBPrep(data).verbatim_latitude_longitude_autocomplete()
#DBPrep(data).coordinate_precision_autocomplete() Deprecated function implemented on GeorefCalculator

In [3]:
data_pep=data.query("decimalLatitude.notna() and decimalLongitude.notna() ")
for index in data_pep.index:
    print(f"register {index+1} of {len(data_pep)}")
    verbatimLatitude=data_pep.loc[index,"decimalLatitude"]
    verbatimLongitude=data_pep.loc[index,"decimalLongitude"]
    dynamicProperties=data_pep.loc[index,"dynamicProperties"]
    verbatimCoordinteSystem="decimal degrees"
    verbatimSRS=data_pep.loc[index,"geodeticDatum"]
    SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).georeferencer()
    result_dict=SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).get_results()
    for keys in result_dict.keys():
        data.loc[index,keys]=result_dict[keys]
    

register 1 of 27
register 2 of 27
register 3 of 27
register 4 of 27
register 5 of 27
register 6 of 27
register 7 of 27
register 8 of 27
register 9 of 27
register 10 of 27
register 11 of 27
register 12 of 27
register 13 of 27
register 14 of 27
register 15 of 27
register 16 of 27
register 17 of 27
register 18 of 27
register 19 of 27
register 20 of 27
register 21 of 27
register 22 of 27
register 23 of 27
register 24 of 27
register 25 of 27
register 26 of 27
register 27 of 27


In [ ]:
data_pep=data.query("verbatimLatitude.notna() and verbatimLongitude.notna() ")

for index in data_pep.index:
    print(f"register {index+1} of {len(data_pep)}")
    verbatimLatitude=data_pep.loc[index,"verbatimLatitude"]
    verbatimLongitude=data_pep.loc[index,"verbatimLongitude"]
    dynamicProperties=data_pep.loc[index,"dynamicProperties"]
    verbatimCoordinteSystem=data_pep.loc[index,"verbatimCoordinateSystem"]
    verbatimSRS=data_pep.loc[index,"verbatimSRS"]
    SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).georeferencer()
    result_dict=SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).get_results()
    for keys in result_dict.keys():
        data.loc[index,keys]=result_dict[keys]
driver.quit()

In [4]:
%%time
# Testing ...
#coord_transform(data).utm_to_dg() Deprecated function implemented on GeorefCalculator
#CoordTransform(data).dms_to_dg() Deprecated function implemented on GeorefCalculator
#CoordTransform(data).geodetic_datum_autocomplete()  Deprecated function implemented on GeorefCalculator
CoordTransform(data).location_autocomplete() 
CoordTransform(data).continent_autocomplete() 
CoordTransform(data).higher_geography_autocomplete()

KeyError: "None of ['country'] are in the columns"

In [5]:
data.to_csv("resultadosGeorrefPeces.csv")

In [ ]:
list1=data.query('minimumElevationInMeters.isnull() and maximumElevationInMeters.isnull() and decimalLatitude.notnull() and decimalLongitude.notnull()').index.tolist()

In [ ]:
CoordTransform(data).elevation_autocomplete() 

In [ ]:
list2=data.query('minimumElevationInMeters.notnull() and maximumElevationInMeters.notnull() and decimalLatitude.notnull() and decimalLongitude.notnull()').index.tolist()

In [ ]:
list(set(list1) - set(list2))

In [ ]:
data.to_csv()